In [1]:
from Bio import Entrez
import urllib
import requests
import re
from bs4 import BeautifulSoup

In [2]:
def get_NCBIresults(ProtAcc):
    Results = {}
    Results['Protein_Name'] = ProtAcc
    Entrez.email = 'adrajesh@ucsd.edu'
    paramEutils = { 'usehistory':'Y' }        # Use Entrez search history to cache results
    try:
        eSearch = Entrez.esearch(db="protein", term=ProtAcc, **paramEutils)
        res = Entrez.read(eSearch)
        ProtID = res['IdList'][0]
    except IndexError:
        print('WARNING: Protein ID unavailable for', ProtAcc)
        Results['Protein_ID'] =None
        Results['Gene_ID'] = None
        Results['Start'] = None
        Results['Assembly_Refseq_ID'] = None
        Results['Assembly_GeneBank_ID'] = None
    else:
        Results['Protein_ID'] =ProtID
        handle = Entrez.efetch(db="protein", id=ProtAcc, rettype="gp", retmode="xml")
        record =Entrez.read(handle)
        handle.close()
        try:
            eSearch2 = Entrez.esearch(db="gene", term=ProtAcc, **paramEutils)
            res2 = Entrez.read(eSearch2)
            GeneID = res2['IdList'][0]
        except IndexError:
            handle = Entrez.efetch(db="protein", id=ProtID, rettype="gp", retmode="text")
            record =handle.read()
            handle.close()
            g = record[record.find("GeneID")::]
            GeneID = g[g.find(":")+1:g.find('\"')]
            if GeneID == '':
                print('WARNING: Gene ID unavailable for', ProtAcc)
                Results['Gene_ID'] = None
                Results['Start'] = None
                Results['Assembly_Refseq_ID'] = None
                Results['Assembly_GeneBank_ID'] = None
            else:
                handle = Entrez.efetch(db="gene", id=GeneID, rettype="gb", retmode="text")
                record =handle.read()
                handle.close()
                Ann = record[record.find("\nAnnotation")+1:record.find(")\n")]
                AID = Ann[Ann.find("N"):Ann.find(".")+2]
                #print(AID)
                Start = int(Ann[Ann.find("(")+1:Ann.find("..")])
                Results['Gene_ID'] = GeneID
                Results['Start'] =Start
                Results['Assembly_Refseq_ID'] = AID
                handle = Entrez.efetch(db="nucleotide", id=AID, rettype="gb", retmode="xml")
                record = Entrez.read(handle)
                handle.close()
                gbID = record[0]['GBSeq_contig']
                gbID = gbID[gbID.find("(")+1:gbID.find(":")]
                Results['Assembly_GeneBank_ID'] = gbID
                handle = Entrez.efetch(db="nucleotide", rettype="gb", retmode="xml", id=gbID)
                record = Entrez.read(handle)
                handle.close()
                genome = record[0]['GBSeq_sequence']
                Results['Sequence'] = genome
        else:
            handle = Entrez.efetch(db="gene", id=GeneID, rettype="gb", retmode="text")
            record =handle.read()
            handle.close()
            Ann = record[record.find("\nAnnotation")+1:record.find(")\n")]
            AID = Ann[Ann.find("N"):Ann.find(".")+2]
            Start = int(Ann[Ann.find("(")+1:Ann.find("..")])
            Results['Gene_ID'] = GeneID
            Results['Start'] =Start
            Results['Assembly_Refseq_ID'] = AID
            handle = Entrez.efetch(db="nucleotide", id=AID, rettype="gb", retmode="xml")
            record = Entrez.read(handle)
            handle.close()
            gbID = record[0]['GBSeq_contig']
            gbID = gbID[gbID.find("(")+1:gbID.find(":")]
            Results['Assembly_GeneBank_ID'] = gbID
            handle = Entrez.efetch(db="nucleotide", rettype="gb", retmode="xml", id=gbID)
            record = Entrez.read(handle)
            handle.close()
            genome = record[0]['GBSeq_sequence']
            Results['Sequence'] = genome
    return Results

In [3]:
All_Proteins = ['YP_353073','CBG73463','NP_268897','YP_002747001','BAF03598','BAG46462','YP_003251752','BAE05705','BAF67264','YP_003880342','YP_001886479','YP_005759947','YP_001376196','NP_470568','YP_006685721','YP_006538656','YP_189066','YP_002736920','FM864213','YP_006082695','YP_003445547','YP_004586821','YP_001089468','YP_005679179','YP_001384783','YP_001392519','YP_005869510','YP_001271396','YP_001646422','YP_002336631','YP_005549228','YP_706485','YP_002804732','YP_003472505']

In [4]:
All_Results = []
for Protein in All_Proteins:
    R = get_NCBIresults(Protein)
    All_Results.append(R)

In [5]:
def get_PHASTERresults(Result):
    RID = Result['Assembly_Refseq_ID']
    start = Result['Start']
    flag=0
    header = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:32.0) Gecko/20100101 Firefox/32.0',}
    url = "http://phaster.ca/phaster_api?acc="+RID
    r  = requests.get(url, headers=header)
    data = r.text
    all_matches = re.findall(r'\d{1,15}-\d{1,15}',data)
    for match in all_matches:
        positions = match.split("-")
        if int(positions[0])<=start and int(positions[1])>=start:
            PhageROI = [int(positions[0]),int(positions[1])]
            print('found_region for',RID)
            Result['Phage_region'] = PhageROI
            flag = 1
    if flag ==0:
        print('something is wrong, No results found for', Result['Protein_Name'],RID)

In [6]:
for i,Result in enumerate(All_Results):
    if Result['Gene_ID']!=None:
        get_PHASTERresults(Result)

found_region for NC_007493.2
something is wrong, No results found for CBG73463 NC_013929.1
found_region for NC_002737.2
found_region for NC_012471.1
found_region for NC_010805.1
found_region for NC_013411.1
found_region for NC_007168.1
found_region for NC_009641.1
found_region for NC_014498.1
found_region for NC_010674.1
found_region for NC_017353.1
found_region for NC_009674.1
found_region for NC_003212.1
found_region for NC_018588.1
found_region for NC_018221.1
found_region for NC_002976.3
found_region for NC_012466.1
found_region for NC_017621.1
found_region for NC_013853.1
found_region for NC_015660.1
found_region for NC_009089.1
found_region for NC_017299.1
found_region for NC_009697.1
something is wrong, No results found for YP_001392519 NC_009699.1
found_region for NC_017486.1
found_region for NC_009513.1
found_region for NC_010184.1
found_region for NC_011658.1
found_region for NC_017191.1
found_region for NC_008268.1
found_region for NC_012563.1
found_region for NC_013893.1


In [7]:
All_Results[0].keys()

dict_keys(['Protein_Name', 'Protein_ID', 'Gene_ID', 'Start', 'Assembly_Refseq_ID', 'Assembly_GeneBank_ID', 'Sequence', 'Phage_region'])

In [8]:
All_Results[0]

{'Protein_Name': 'YP_353073',
 'Protein_ID': '552535759',
 'Gene_ID': '3720257',
 'Start': 1704511,
 'Assembly_Refseq_ID': 'NC_007493.2',
 'Assembly_GeneBank_ID': 'CP000143.2',
 'Sequence': '...',
 'Phage_region': [1667836, 1706259]}

In [9]:
#Code to find new Protein ID in removed records:

In [10]:
header = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:32.0) Gecko/20100101 Firefox/32.0',}
url1 = "https://www.ncbi.nlm.nih.gov/protein/YP_003880342"

r1  = requests.get(url1, headers=header)

data1 = r1.text

In [11]:
soup2 = BeautifulSoup(data1)

for link in soup2.find_all("em",class_="detail"):
    string = str(link)
print(string)

<em class="detail">The sequence YP_003880342 is 100% identical to WP_000633509.1 over  its full length. Be aware that a NCBI nonredundant RefSeq protein (WP_) 
                            can be annotated on large numbers of bacterial genomes 
                            that encode that identical protein.<p><a href="/protein/YP_003880342.1?report=genpept">Old YP_003880342.1</a><span>   </span><a href="/protein/WP_000633509.1?">New WP_000633509.1</a><span>   </span><a href="/protein/WP_000633509.1?report=ipg">Identical proteins</a><span>   </span><a href="/refseq/about/prokaryotes/reannotation/">Re-annotation project</a></p></em>


In [12]:
newID = string[string.find("identical to ")+13:string.find(" over")]
newID

'WP_000633509.1'